In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import csv

# Set up Chrome options
options = Options()
options.headless = False  # Set to True for silent operation

# Specify the path to ChromeDriver
service = Service(r'C:\Users\Badrinath\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open IMDb Top 250 page
url = 'https://www.imdb.com/chart/top/'
driver.get(url)

# Wait for the page to load
time.sleep(3)

# Get the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the movie table
movie_table = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 iyTDQy compact-list-view ipc-metadata-list--base')
movies = movie_table.find_all('li', class_='ipc-metadata-list-summary-item sc-4929eaf6-0 DLYcv cli-parent')

# Write data to CSV
csv_filename = "Top_250_IMDB_movies_with_details.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Rank', 'Name', 'Rating', 'Year', 'Duration', 'PG Rating', 'Description', 'Director', 'Writers', 'Stars'])

    for movie in movies:
        # Rank
        rank = movie.find('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-a69a4297-2 bqNXEn cli-title with-margin').a.get_text(strip=True).split('.')[0]
        print(f"Rank: {rank}")

        # Name
        name = movie.find('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-a69a4297-2 bqNXEn cli-title with-margin').a.get_text(strip=True).split('.')[1]
        print(f"Name: {name}")

        # Rating
        rating = movie.find('span', class_='ipc-rating-star--rating').text
        print(f"Rating: {rating}")

        # Metadata (Year, Duration, PG Rating)
        metadata_div = movie.find('div', class_='sc-6ade9358-6 cBtpuV cli-title-metadata')
        metadata_spans = metadata_div.find_all('span', class_='sc-6ade9358-7 exckou cli-title-metadata-item')

        year = metadata_spans[0].text if len(metadata_spans) > 0 else "N/A"
        duration = metadata_spans[1].text if len(metadata_spans) > 1 else "N/A"
        pg_rating = metadata_spans[2].text if len(metadata_spans) > 2 else "N/A"
        
        print(f"Year: {year}, Duration: {duration}, PG Rating: {pg_rating}")

        # Navigate to the individual movie page
        movie_link = "https://www.imdb.com" + movie.find('div', class_='sc-6ade9358-0 ktYEKX cli-children').a['href']
        driver.get(movie_link)
        time.sleep(2)
        
        print(f"Movie Link: {movie_link}")

        # Extract details from the movie page
        movie_soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Description
        description_tag = movie_soup.find('span', attrs={'data-testid': 'plot-xs_to_m'})
        description = description_tag.get_text(strip=True) if description_tag else "N/A"

        # Director
        director_tag = movie_soup.find('li', attrs={'data-testid': 'title-pc-principal-credit'})
        director = director_tag.get_text(strip=True) if director_tag else "N/A"

        # Writers
        writers_tags = movie_soup.find_all('li', attrs={'data-testid': 'title-pc-principal-credit'})[1:3]
        writers = ', '.join([writer.get_text(strip=True) for writer in writers_tags]) if writers_tags else "N/A"

        # Stars
        stars_section = movie_soup.find('li', attrs={'data-testid': 'title-pc-principal-credit'})
        stars_tags = stars_section.find_all('a') if stars_section else []
        stars = ', '.join([star.get_text(strip=True) for star in stars_tags[:3]])  # Get top 3 stars
        
        print(f"Description: {description}")
        print(f"Director: {director}")
        print(f"Writers: {writers}")
        print(f"Stars: {stars}")

        # Write row
        writer.writerow([rank, name, rating, year, duration, pg_rating, description, director, writers, stars,])

# Close the browser
driver.quit()

print(f"Data has been written to {csv_filename}")


Rank: 1
Name:  The Shawshank Redemption
Rating: 9.3
Year: 1994, Duration: 2h 22m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0111161/?ref_=chttp_t_1
Description: A banker convicted of uxoricide forms a friendship over a quarter century with a hardened convict, while maintaining his innocence and trying to remain hopeful through simple compassion.
Director: DirectorFrank Darabont
Writers: WritersStephen KingFrank Darabont, StarsTim RobbinsMorgan FreemanBob Gunton
Stars: Frank Darabont
Rank: 2
Name:  The Godfather
Rating: 9.2
Year: 1972, Duration: 2h 55m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0068646/?ref_=chttp_t_2
Description: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
Director: DirectorFrancis Ford Coppola
Writers: WritersMario PuzoFrancis Ford Coppola, StarsMarlon BrandoAl PacinoJames Caan
Stars: Francis Ford Coppola
Rank: 3
Name:  The Dark Knight
Rating: 9.0
Year: 2008, Duration: 2h 3

Movie Link: https://www.imdb.com/title/tt0073486/?ref_=chttp_t_18
Description: In the Fall of 1963, a Korean War veteran and criminal pleads insanity and is admitted to a mental institution, where he rallies up the scared patients against the tyrannical nurse.
Director: DirectorMilos Forman
Writers: WritersLawrence HaubenBo GoldmanKen Kesey, StarsJack NicholsonLouise FletcherMichael Berryman
Stars: Milos Forman
Rank: 19
Name:  Interstellar
Rating: 8.7
Year: 2014, Duration: 2h 49m, PG Rating: PG-13
Movie Link: https://www.imdb.com/title/tt0816692/?ref_=chttp_t_19
Description: When Earth becomes uninhabitable in the future, a farmer and ex-NASA pilot, Joseph Cooper, is tasked to pilot a spacecraft, along with a team of researchers, to find a new planet for humans.
Director: DirectorChristopher Nolan
Writers: WritersJonathan NolanChristopher Nolan, StarsMatthew McConaugheyAnne HathawayJessica Chastain
Stars: Christopher Nolan
Rank: 20
Name:  Se7en
Rating: 8.6
Year: 1995, Duration: 2h 7m, 

Movie Link: https://www.imdb.com/title/tt0110357/?ref_=chttp_t_36
Description: Lion prince Simba and his father are targeted by his bitter uncle, who wants to ascend the throne himself.
Director: DirectorsRoger AllersRob Minkoff
Writers: WritersIrene MecchiJonathan RobertsLinda Woolverton, StarsMatthew BroderickJeremy IronsJames Earl Jones
Stars: Roger Allers, Rob Minkoff
Rank: 37
Name:  The Departed
Rating: 8.5
Year: 2006, Duration: 2h 31m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0407887/?ref_=chttp_t_37
Description: An undercover cop and a mole in the police attempt to identify each other while infiltrating an Irish gang in South Boston.
Director: DirectorMartin Scorsese
Writers: WritersWilliam MonahanAlan MakFelix Chong, StarsLeonardo DiCaprioMatt DamonJack Nicholson
Stars: Martin Scorsese
Rank: 38
Name:  Spider-Man: Across the Spider-Verse
Rating: 8.6
Year: 2023, Duration: 2h 20m, PG Rating: PG
Movie Link: https://www.imdb.com/title/tt9362722/?ref_=chttp_t_38
Descript

Description: A mysterious stranger with a harmonica joins forces with a notorious desperado to protect a beautiful widow from a ruthless assassin working for the railroad.
Director: DirectorSergio Leone
Writers: WritersSergio DonatiSergio LeoneDario Argento, StarsHenry FondaCharles BronsonClaudia Cardinale
Stars: Sergio Leone
Rank: 54
Name:  Django Unchained
Rating: 8.5
Year: 2012, Duration: 2h 45m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt1853728/?ref_=chttp_t_54
Description: With the help of a German bounty-hunter, a freed slave sets out to rescue his wife from a brutal plantation owner in Mississippi.
Director: DirectorQuentin Tarantino
Writers: WriterQuentin Tarantino, StarsJamie FoxxChristoph WaltzLeonardo DiCaprio
Stars: Quentin Tarantino
Rank: 55
Name:  City Lights
Rating: 8.5
Year: 1931, Duration: 1h 27m, PG Rating: G
Movie Link: https://www.imdb.com/title/tt0021749/?ref_=chttp_t_55
Description: With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has f

Description: In Nazi-occupied France during World War II, a plan to assassinate Nazi leaders by a group of Jewish U.S. soldiers coincides with a theatre owner's vengeful plans for the same.
Director: DirectorQuentin Tarantino
Writers: WriterQuentin Tarantino, StarsBrad PittDiane KrugerEli Roth
Stars: Quentin Tarantino
Rank: 71
Name:  The Dark Knight Rises
Rating: 8.4
Year: 2012, Duration: 2h 44m, PG Rating: PG-13
Movie Link: https://www.imdb.com/title/tt1345836/?ref_=chttp_t_71
Description: Bane, an imposing terrorist, attacks Gotham City and disrupts its eight-year-long period of peace. This forces Bruce Wayne to come out of hiding and don the cape and cowl of Batman again.
Director: DirectorChristopher Nolan
Writers: WritersJonathan NolanChristopher NolanDavid S. Goyer, StarsChristian BaleTom HardyAnne Hathaway
Stars: Christopher Nolan
Rank: 72
Name:  Coco
Rating: 8.4
Year: 2017, Duration: 1h 45m, PG Rating: PG
Movie Link: https://www.imdb.com/title/tt2380307/?ref_=chttp_t_72
Descrip

...

Movie Link: https://www.imdb.com/title/tt0116282/?ref_=chttp_t_175
Description: Minnesota car salesman Jerry Lundegaard's inept crime falls apart due to his and his henchmen's bungling and the persistent police work of the quite pregnant Marge Gunderson.
Director: DirectorsJoel CoenEthan Coen
Writers: WritersEthan CoenJoel Coen, StarsWilliam H. MacyFrances McDormandSteve Buscemi
Stars: Joel Coen, Ethan Coen
Rank: 176
Name:  Warrior
Rating: 8.1
Year: 2011, Duration: 2h 20m, PG Rating: PG-13
Movie Link: https://www.imdb.com/title/tt1291584/?ref_=chttp_t_176
Description: The youngest son of an alcoholic former boxer returns home, where he's trained by his father for competition in a mixed martial arts tournament - a path that puts the fighter on a collision ...Read all
Director: DirectorGavin O'Connor
Writers: WritersGavin O'ConnorAnthony TambakisCliff Dorfman, StarsTom HardyNick NolteJoel Edgerton
Stars: Gavin O'Connor
Rank: 177
Name:  Spider-Man: No Way Home
Rating: 8.2
Year: 2021, Dura

Movie Link: https://www.imdb.com/title/tt2267998/?ref_=chttp_t_192
Description: The husband of a missing woman becomes the chief suspect in her disappearance.
Director: DirectorDavid Fincher
Writers: WriterGillian Flynn, StarsBen AffleckRosamund PikeNeil Patrick Harris
Stars: David Fincher
Rank: 193
Name:  Memories of Murder
Rating: 8.1
Year: 2003, Duration: 2h 12m, PG Rating: Not Rated
Movie Link: https://www.imdb.com/title/tt0353969/?ref_=chttp_t_193
Description: In a small Korean province in 1986, two detectives struggle with the case of multiple young women being found raped and murdered by an unknown culprit.
Director: DirectorBong Joon Ho
Writers: WritersBong Joon HoKwang-rim KimSung-bo Shim, StarsSong Kang-hoKim Sang-kyungKim Roe-ha
Stars: Bong Joon Ho
Rank: 194
Name:  In the Name of the Father
Rating: 8.1
Year: 1993, Duration: 2h 13m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0107207/?ref_=chttp_t_194
Description: An Irish man's coerced confession to an I.R.A. bombi

Movie Link: https://www.imdb.com/title/tt0050986/?ref_=chttp_t_209
Description: After living a life marked by coldness, an aging professor is forced to confront the emptiness of his existence.
Director: DirectorIngmar Bergman
Writers: WritersIngmar BergmanLouis Sauvat, StarsVictor SjöströmBibi AnderssonIngrid Thulin
Stars: Ingmar Bergman
Rank: 210
Name:  Mr
Rating: 8.1
Year: 1939, Duration: 2h 9m, PG Rating: Approved
Movie Link: https://www.imdb.com/title/tt0031679/?ref_=chttp_t_210
Description: A naive youth leader is appointed to fill a vacancy in the U.S. Senate. His idealistic plans promptly collide with corruption at home and subterfuge from his hero in Washington, but he tries...Read all
Director: DirectorFrank Capra
Writers: WritersSidney BuchmanLewis R. FosterMyles Connolly, StarsJames StewartJean ArthurClaude Rains
Stars: Frank Capra
Rank: 211
Name:  The Third Man
Rating: 8.1
Year: 1949, Duration: 1h 33m, PG Rating: Approved
Movie Link: https://www.imdb.com/title/tt0041959/?re

Movie Link: https://www.imdb.com/title/tt0036868/?ref_=chttp_t_227
Description: Three World War II veterans, two of them traumatized or disabled, return home to the American midwest to discover that they and their families have been irreparably changed.
Director: DirectorWilliam Wyler
Writers: WritersRobert E. SherwoodMacKinlay Kantor, StarsMyrna LoyDana AndrewsFredric March
Stars: William Wyler
Rank: 228
Name:  The Exorcist
Rating: 8.1
Year: 1973, Duration: 2h 2m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt0070047/?ref_=chttp_t_228
Description: When a mysterious entity possesses a young girl, her mother seeks the help of two Catholic priests to save her life.
Director: DirectorWilliam Friedkin
Writers: WriterWilliam Peter Blatty, StarsEllen BurstynMax von SydowLinda Blair
Stars: William Friedkin
Rank: 229
Name:  Rush
Rating: 8.1
Year: 2013, Duration: 2h 3m, PG Rating: R
Movie Link: https://www.imdb.com/title/tt1979320/?ref_=chttp_t_229
Description: James Hunt and Niki Lauda

Movie Link: https://www.imdb.com/title/tt0245712/?ref_=chttp_t_245
Description: An amateur dog fighter, a supermodel, and a derelict assassin, all separately struggling to find love, find their lives transformed by a devastating car wreck in Mexico City.
Director: DirectorAlejandro G. Iñárritu
Writers: WriterGuillermo Arriaga, StarsEmilio EchevarríaGael García BernalGoya Toledo
Stars: Alejandro G. Iñárritu
Rank: 246
Name:  Rebecca
Rating: 8.1
Year: 1940, Duration: 2h 10m, PG Rating: Approved
Movie Link: https://www.imdb.com/title/tt0032976/?ref_=chttp_t_246
Description: A self-conscious woman juggles adjusting to her new role as an aristocrat's wife and avoiding being intimidated by his first wife's spectral presence.
Director: DirectorAlfred Hitchcock
Writers: WritersDaphne Du MaurierRobert E. SherwoodJoan Harrison, StarsLaurence OlivierJoan FontaineGeorge Sanders
Stars: Alfred Hitchcock
Rank: 247
Name:  The Help
Rating: 8.1
Year: 2011, Duration: 2h 26m, PG Rating: PG-13
Movie Link: h